In [2]:
import numpy as np
a = np.array([ [100],
               [101],
               [102],
               [103],
               [104],
               [105],
               [106],
               [107],
               [108],
               [109]])

a.take([1,2,3], axis=0)

array([[101],
       [102],
       [103]])

In [30]:
from keras import models, layers, optimizers
model = models.Sequential()
model.add(layers.Dense(3, activation='relu', input_shape=(3,)))
model.add(layers.Dense(3, activation='sigmoid'))#, kernel_initializer='zeros', bias_initializer='zeros'))
model.compile(optimizers.Adam(lr=0.01),loss='mse')

In [31]:
import numpy as np
#model.fit(np.array([[1,2,5]]),np.array([[1,1,1]]), epochs=40)
model.predict(np.array([1,2,5]).reshape(-1,3))

array([[0.49996555, 0.49831614, 0.5038847 ]], dtype=float32)

In [17]:
num_envs = 10
NotDisplay=False
velocity_alpha=0.01
env_name="CartPole-v1"
p_env = ParallelEnvironment(env_name, num_envs,NotDisplay)

params=[]
best_params=[]
best_returns=[]
best_pos=0
velocity=[]
iter_cnt=0
inertia=1
cognitive=1
social=1

for i in range(num_envs):
    params.append(np.random.uniform(-1,1,4))
    best_returns.append(0)
    best_params.append(np.array([0 for i in range(4)]))
    velocity.append(np.array([0.0 for i in range(4)]))

while True:
    returns=p_env.episode(params)
    iter_cnt+=1

    #Output of the batch episode.
    print ("Number of batch episodes ran -> ",iter_cnt)
    print ("Parameter for the batch for last episode ->") 
    print (np.around(params,3))
    print ("Best Parameters for the batch for all episodes ->")
    print (np.around(best_params,3))
    print ("Returns for the batch for last episode -> ",returns)
    print ("Returns for the batch for all episodes -> ",best_returns)
    print ("Rate of change of parameters for the batch -> ")
    print (np.around(velocity,3))

    #Exit condition
    if returns==best_returns:
        print ("Batch converged after {} iterations.".format(iter_cnt))
        p_env.__del__()

        # Run a final episode using the average of all the final parameters
        p_start, p_end = Pipe()
        env_worker = EnvWorker("CartPole-v1", p_end, name="Worker",NotDisplay=True,delay=0.02)
        env_worker.start()
        p_start.send(np.sum(best_params,axis=0)/num_envs)
        episode_return = p_start.recv()
        print ("Reward for the final episode ->", episode_return)
        p_start.send("EXIT")
        env_worker.terminate()

    # update according to PSO algorithm		
    for i in range(num_envs):
        if(returns[i]>=best_returns[i]):
            best_returns[i]=returns[i]
            best_params[i]=params[i]

    best_pos=returns.index(max(returns))

    for i in range(num_envs):
        velocity[i]=(inertia*velocity[i]
            +cognitive*np.random.uniform(0,velocity_alpha)*(best_params[i]-params[i])
            +social*np.random.uniform(0,velocity_alpha)*(best_params[best_pos]-params[i]))
        params[i]+=velocity[i]
    pass

NameError: name 'ParallelEnvironment' is not defined